In [9]:
import argparse
import gym
import numpy as np
from itertools import count

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical



seed=543
gamma = 0.99
render = 'store_true'
log_interval = 10

env = gym.make('CartPole-v0')
env.seed(seed)
torch.manual_seed(seed)


class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(4, 128)
        self.affine2 = nn.Linear(128, 2)

        self.saved_log_probs = []
        self.rewards = []

    def forward(self, x):
        x = F.relu(self.affine1(x))
        action_scores = self.affine2(x)
        return F.softmax(action_scores, dim=1)


policy = Policy()
optimizer = optim.Adam(policy.parameters(), lr=1e-2)
eps = np.finfo(np.float32).eps.item()


def select_action(state):
    state = torch.from_numpy(state).float().unsqueeze(0)
    probs = policy(state)
    print(probs)
    m = Categorical(probs)
    action = m.sample()
    policy.saved_log_probs.append(m.log_prob(action))
    return action.item()


def finish_episode():
    R = 0
    policy_loss = []
    rewards = []
    for r in policy.rewards[::-1]:
        R = r + gamma * R
        rewards.insert(0, R)
    rewards = torch.tensor(rewards)
    rewards = (rewards - rewards.mean()) / (rewards.std() + eps)
    for log_prob, reward in zip(policy.saved_log_probs, rewards):
        policy_loss.append(-log_prob * reward)
    optimizer.zero_grad()
    policy_loss = torch.cat(policy_loss).sum()
    policy_loss.backward()
    optimizer.step()
    del policy.rewards[:]
    del policy.saved_log_probs[:]


def main():
    running_reward = 10
    for i_episode in count(1):
        state = env.reset()
        for t in range(10000):  # Don't infinite loop while learning
            action = select_action(state)
            state, reward, done, _ = env.step(action)
            if render:
                env.render()
            policy.rewards.append(reward)
            if done:
                break

        running_reward = running_reward * 0.99 + t * 0.01
        finish_episode()
        if i_episode % log_interval == 0:
            print('Episode {}\tLast length: {:5d}\tAverage length: {:.2f}'.format(
                i_episode, t, running_reward))
        if running_reward > env.spec.reward_threshold:
            print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(running_reward, t))
            break

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [10]:
state = env.reset()

In [12]:
select_action(state)

tensor([[0.5007, 0.4993]], grad_fn=<SoftmaxBackward>)


0

In [3]:
main()

Episode 10	Last length:     8	Average length: 10.45
Episode 20	Last length:    11	Average length: 10.68
Episode 30	Last length:     8	Average length: 10.62
Episode 40	Last length:    10	Average length: 10.60
Episode 50	Last length:    28	Average length: 10.84
Episode 60	Last length:    24	Average length: 13.61
Episode 70	Last length:    14	Average length: 13.93
Episode 80	Last length:    16	Average length: 13.95
Episode 90	Last length:    24	Average length: 14.21
Episode 100	Last length:    14	Average length: 14.57
Episode 110	Last length:    10	Average length: 14.58
Episode 120	Last length:     9	Average length: 14.00
Episode 130	Last length:     9	Average length: 13.47
Episode 140	Last length:     8	Average length: 12.99
Episode 150	Last length:     9	Average length: 12.52
Episode 160	Last length:     9	Average length: 12.16
Episode 170	Last length:     8	Average length: 11.84
Episode 180	Last length:     8	Average length: 11.52
Episode 190	Last length:    10	Average length: 11.22
Ep

Episode 1550	Last length:   121	Average length: 151.43
Episode 1560	Last length:   118	Average length: 147.71
Episode 1570	Last length:   115	Average length: 144.93
Episode 1580	Last length:   119	Average length: 142.64
Episode 1590	Last length:   116	Average length: 140.43
Episode 1600	Last length:   111	Average length: 138.83
Episode 1610	Last length:    63	Average length: 134.69
Episode 1620	Last length:    67	Average length: 128.85
Episode 1630	Last length:    41	Average length: 122.21
Episode 1640	Last length:    56	Average length: 115.48
Episode 1650	Last length:    58	Average length: 109.44
Episode 1660	Last length:    28	Average length: 103.17
Episode 1670	Last length:    53	Average length: 98.55
Episode 1680	Last length:    73	Average length: 95.52
Episode 1690	Last length:   138	Average length: 96.82
Episode 1700	Last length:   191	Average length: 103.63
Episode 1710	Last length:   199	Average length: 112.59
Episode 1720	Last length:   199	Average length: 120.85
Episode 1730	

Episode 3050	Last length:    96	Average length: 130.31
Episode 3060	Last length:    89	Average length: 126.83
Episode 3070	Last length:    99	Average length: 123.97
Episode 3080	Last length:    91	Average length: 120.91
Episode 3090	Last length:    89	Average length: 117.65
Episode 3100	Last length:    81	Average length: 114.25
Episode 3110	Last length:    89	Average length: 111.67
Episode 3120	Last length:    94	Average length: 108.97
Episode 3130	Last length:    82	Average length: 106.67
Episode 3140	Last length:    85	Average length: 104.23
Episode 3150	Last length:    85	Average length: 102.17
Episode 3160	Last length:    72	Average length: 99.94
Episode 3170	Last length:    83	Average length: 98.24
Episode 3180	Last length:    77	Average length: 96.75
Episode 3190	Last length:    87	Average length: 95.60
Episode 3200	Last length:    91	Average length: 95.00
Episode 3210	Last length:    90	Average length: 95.17
Episode 3220	Last length:    93	Average length: 95.19
Episode 3230	Last

Episode 4550	Last length:   172	Average length: 155.60
Episode 4560	Last length:   196	Average length: 158.70
Episode 4570	Last length:   199	Average length: 161.45
Episode 4580	Last length:   167	Average length: 164.23
Episode 4590	Last length:   199	Average length: 167.45
Episode 4600	Last length:   199	Average length: 170.40
Episode 4610	Last length:   199	Average length: 173.09
Episode 4620	Last length:   199	Average length: 175.56
Episode 4630	Last length:   108	Average length: 175.88
Episode 4640	Last length:   199	Average length: 175.86
Episode 4650	Last length:   199	Average length: 174.70
Episode 4660	Last length:   179	Average length: 176.37
Episode 4670	Last length:   156	Average length: 177.31
Episode 4680	Last length:   199	Average length: 179.38
Episode 4690	Last length:   188	Average length: 180.03
Episode 4700	Last length:   199	Average length: 180.29
Episode 4710	Last length:   199	Average length: 179.49
Episode 4720	Last length:    81	Average length: 176.24
Episode 47

In [3]:
dict([1,2,3])

3

In [ ]:
plt.title('hello')
plt.title = 'hello'
